In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

# Use a white background for matplotlib figures
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
from torchvision.transforms import ToTensor
dataset = MNIST(root='data/',download=False, transform = ToTensor())

In [ ]:
img, label = dataset[0]
print(img[0:2])

In [ ]:

print(img.shape)
print(label)
plt.imshow(img.reshape((28, 28)), cmap='gray')

In [ ]:
from torch.utils.data import random_split
val_size = 10_000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

In [ ]:
from torch.utils.data.dataloader import DataLoader
btch = 128
train_dl = DataLoader(train_ds, btch, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, btch, num_workers=4, pin_memory=True)


In [ ]:
from torchvision.utils import make_grid
for imgs, lbl in train_dl:
    print(img.shape)
    plt.figure(figsize=(16, 8))
    plt.axis('off')
    plt.imshow(make_grid(imgs, nrow=16).permute((1, 2, 0)))
    break

In [ ]:
for imgs, _ in train_dl:
    inputs = imgs.reshape(-1,(28*28)) # =>(btch,img.flatten)
    print(inputs.shape)
    break

In [ ]:
for imgs, _ in train_dl:
    inputs = imgs.reshape(-1,(28*28)) # =>(btch,img.flatten)
    print(inputs)
    break

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class MNISTModel(nn.Module):
    def __init__(self,input_neurons,hidden_neurons, output):
        super().__init__()
        self.lyr_1 = nn.Linear(input_neurons, 128)
        self.lyr_hidden = nn.Linear(128, hidden_neurons) #added
        self.lyr_2 = nn.Linear(hidden_neurons, output)

    def forward(self, xbtch):
        xbtch = xbtch.view(xbtch.size(0), -1) 
        out = self.lyr_1(xbtch)
        out = F.relu(out) #apply relu on the output
        out = self.lyr_hidden(out)
        out = F.relu(out)
        out = self.lyr_2(out)
        return out

    def training_step(self, btch):
        imgs, labels = btch
        out = self(imgs) #run the forward method on the images
        loss = F.cross_entropy(out, labels)
        return loss

    def validation_step(self, btch):
        imgs, labels = btch
        out =self(imgs)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_acc':acc, 'val_loss':loss}

    
    def validation_epoch_end(self, outputs):
        btch_losses = [x['val_loss'] for x in outputs]
        btch_acc = [x['val_acc'] for x in outputs]
        # combine the losses & accuracies
        epoch_loss = torch.stack(btch_losses).mean() 
        epoch_acc = torch.stack(btch_acc).mean()
        return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item()}

    def epoch_end(self, epoch, results):
        print(f"Epoch [{epoch}], validation_loss: {results['val_loss']:4f}, valdation_acc: {results['val_acc']:4f}")
    


In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
input_neurons = 28*28
hidden_size = 64 #! added was 32
num_classes= 10 #labels 

In [ ]:
model = MNISTModel(input_neurons, hidden_size, num_classes)

# USING GPUS IF AVALIABLE

In [ ]:
def SetGPU():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
device = SetGPU()
device

In [ ]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for btch in self.dl:
            yield to_device(btch, self.device)
    def __len__(self):
        return len(dl)

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
vdl = DeviceDataLoader(val_dl, device)

In [ ]:
def evaluate(model,val_loader):
    outputs = [model.validation_step(btch) for btch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
def fit(model, epochs, lr, train_loader,val_loader,opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        #training phase
        for btch in train_loader:
            loss = model.training_step(btch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        #validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
to_device(model, device)

In [ ]:
history = [evaluate(model,vdl)]
history

In [ ]:
history += fit(model, 30, 5e-2, train_dl, vdl)

In [ ]:
history += fit(model, 30, 1e-3, train_dl, vdl)

In [ ]:
history += fit(model, 30, 1e-4, train_dl, vdl)

In [ ]:
import os
os.listdir()

In [ ]:
# torch.save(model, 'drive/MyDrive/programming_project/digitLinearModel.pth')
torch.save(model.state_dict(), 'drive/MyDrive/programming_project/digit_state_dict.pth')